In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import re

In [3]:
school_support = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\School Support.csv")
enrollment = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Enrollment.csv")
corporal_punishment = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Corporal Punishment.csv")
expulsions = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Expulsions.csv")

C:\Users\Omkaar Sampigeadi\AppData\Local\Temp\ipykernel_23808\1544074868.py:1: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  school_support = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\School Support.csv")
C:\Users\Omkaar Sampigeadi\AppData\Local\Temp\ipykernel_23808\1544074868.py:2: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  enrollment = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Enrollment.csv")
C:\Users\Omkaar Sampigeadi\AppData\Local\Temp\ipykernel_23808\1544074868.py:3: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  corporal_punishment = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Corporal Punishment.csv")
C:\Users\Omkaar Sampigeadi\AppData\Local\Temp\ipykernel_238

In [19]:
# Descriptive statistics for key metrics
def analyze_descriptive_stats(df, metric_columns, table_heading):
    """Generate descriptive statistics for specified columns"""
    stats_summary = df[metric_columns].describe()
    print(f"Descriptive Statistics: {table_heading}")
    print(stats_summary)
    print()


# Example for school support data
support_metrics = ['SCH_FTETEACH_TOT', 'SCH_FTETEACH_CERT', 'SCH_FTETEACH_NOTCERT']
if all(col in school_support.columns for col in support_metrics):
   analyze_descriptive_stats(school_support, support_metrics, "School Support Metrics")


# Example for enrollment data - focusing on disability-related columns
disability_metrics = [
    'SCH_ENR_IDEA_M',     # Male students served under IDEA (Individuals with Disabilities Education Act)
    'SCH_ENR_IDEA_F',     # Female students served under IDEA (Individuals with Disabilities Education Act
    'SCH_ENR_504_M',      # Male students with 504 plans (Section 504 of the Rehabilitation Act)
    'SCH_ENR_504_F'       # Female students with 504 plans (Section 504 of the Rehabilitation Act)
]

analyze_descriptive_stats(enrollment, disability_metrics, "Enrollment Metrics for Students with Disabilities")

# Descriptive analysis for corporal punishment dataset
corporal_disability_metrics = [
  key for key in corporal_punishment.keys() if "_DISCWDIS_" in key
]

# Check which columns exist in the dataset
existing_corporal_cols = [col for col in corporal_disability_metrics if col in corporal_punishment.columns]
if existing_corporal_cols:
    analyze_descriptive_stats(corporal_punishment, existing_corporal_cols,
                              "Corporal Punishment for Students with Disabilities")
else:
    print("No corporal punishment disability columns found in dataset")

# Descriptive analysis for expulsions dataset among IDEA students
expulsion_disability_metrics = [
    "SCH_PSDISC_EXP_IDEA_M",  #
    "SCH_PSDISC_EXP_IDEA_F",
]

# Check which columns exist in the dataset
existing_expulsion_cols = [col for col in expulsion_disability_metrics if col in expulsions.columns]

analyze_descriptive_stats(expulsions, existing_expulsion_cols, "Expulsions for Students with Disabilities")
if len(existing_expulsion_cols) == 0:
    print("No expulsion disability columns found in dataset")
# First 5 columns

Descriptive Statistics: School Support Metrics
       SCH_FTETEACH_TOT  SCH_FTETEACH_CERT  SCH_FTETEACH_NOTCERT
count      98010.000000       98010.000000          98010.000000
mean          34.605761          33.161289              1.442351
std           27.210973          26.455871              4.497697
min           -6.000000          -6.000000             -6.000000
25%           18.010000          17.000000              0.000000
50%           29.330000          28.000000              0.000000
75%           43.430000          42.000000              1.000000
max         1074.000000         784.000000            305.000000

Descriptive Statistics: Enrollment Metrics for Students with Disabilities
       SCH_ENR_IDEA_M  SCH_ENR_IDEA_F  SCH_ENR_504_M  SCH_ENR_504_F
count    98010.000000    98010.000000   98010.000000   98010.000000
mean        44.636700       23.561912       7.944904       5.245975
std         44.503713       25.511122      19.941082      16.584246
min         -9.000000

In [22]:

# Function to filter columns using regex patterns
def filter_columns_by_regex(df, pattern, description):
    """Filter dataframe columns using regex pattern"""
    matching_cols = [col for col in df.columns if re.search(pattern, col, re.IGNORECASE)]
    print(f"{description}: {len(matching_cols)} columns found")
    if matching_cols:
        print(f"Sample columns: {matching_cols[:3]}")
    return matching_cols


# Filter by gender (Male/Female)
print("=== FILTERING BY GENDER ===")
male_disability_cols = filter_columns_by_regex(enrollment, r'(IDEA|504).*_M$', "Male students with disabilities")
female_disability_cols = filter_columns_by_regex(enrollment, r'(IDEA|504).*_F$', "Female students with disabilities")

# Analyze by gender
if male_disability_cols:
    analyze_descriptive_stats(enrollment, male_disability_cols, "Male Students with Disabilities")
if female_disability_cols:
    analyze_descriptive_stats(enrollment, female_disability_cols, "Female Students with Disabilities")

# Filter by race/ethnicity using regex
print("=== FILTERING BY RACE/ETHNICITY ===")
race_patterns = {
    'Hispanic': r'_HI_[MF]$',
    'American Indian': r'_AM_[MF]$',
    'Asian': r'_AS_[MF]$',
    'Hawaiian/Pacific Islander': r'_HP_[MF]$',
    'Black': r'_BL_[MF]$',
    'White': r'_WH_[MF]$',
    'Two or More Races': r'_TR_[MF]$'
}

# Filter enrollment data by race and disability status
for race, pattern in race_patterns.items():
    disability_race_cols = filter_columns_by_regex(enrollment, f'(IDEA|504).*{pattern}',
                                                   f"{race} students with disabilities")

# Filter disciplinary data by race and disability
print("=== CORPORAL PUNISHMENT BY RACE AND DISABILITY ===")
for race, pattern in race_patterns.items():
    corporal_race_cols = filter_columns_by_regex(corporal_punishment, f'DISCWDIS.*{pattern}',
                                                 f"{race} students - corporal punishment")

print("=== EXPULSIONS BY RACE AND DISABILITY ===")
for race, pattern in race_patterns.items():
    expulsion_race_cols = filter_columns_by_regex(expulsions, f'(PSDISC|DISCWDIS).*{pattern}',
                                                  f"{race} students - expulsions")


# Filter by disability type (IDEA vs 504 plans)
print("=== FILTERING BY DISABILITY TYPE ===")
idea_enrollment = filter_columns_by_regex(enrollment, r'IDEA_[MF]$', "IDEA students")
section504_enrollment = filter_columns_by_regex(enrollment, r'504_[MF]$', "Section 504 students")


# Targeted analysis: Black male students with disabilities
print("=== TARGETED ANALYSIS: BLACK MALE STUDENTS WITH DISABILITIES ===")
black_male_disability = filter_columns_by_regex(enrollment, r'(IDEA|504).*_BL_M$',
                                                "Black male students with disabilities")
if black_male_disability:
    analyze_descriptive_stats(enrollment, black_male_disability, "Black Male Students with Disabilities - Enrollment")

# Compare with disciplinary actions
black_male_corporal = filter_columns_by_regex(corporal_punishment, r'DISCWDIS.*_BL_M$',
                                              "Black male students - corporal punishment")
black_male_expulsions = filter_columns_by_regex(expulsions, r'(PSDISC|DISCWDIS).*_BL_M$',
                                                "Black male students - expulsions")

if black_male_corporal:
    analyze_descriptive_stats(corporal_punishment, black_male_corporal,
                              "Black Male Students with Disabilities - Corporal Punishment")
if black_male_expulsions:
    analyze_descriptive_stats(expulsions, black_male_expulsions, "Black Male Students with Disabilities - Expulsions")

=== FILTERING BY GENDER ===
Male students with disabilities: 40 columns found
Sample columns: ['SCH_ENR_IDEA_M', 'SCH_ENR_504_M', 'SCH_IDEAENR_HI_M']
Female students with disabilities: 40 columns found
Sample columns: ['SCH_ENR_IDEA_F', 'SCH_ENR_504_F', 'SCH_IDEAENR_HI_F']
Descriptive Statistics: Male Students with Disabilities
       SCH_ENR_IDEA_M  SCH_ENR_504_M  SCH_IDEAENR_HI_M  SCH_IDEAENR_AM_M  \
count    98010.000000   98010.000000      98010.000000      98010.000000   
mean        44.636700       7.944904         12.460351          0.155913   
std         44.503713      19.941082         22.645671          3.373580   
min         -9.000000      -9.000000         -9.000000         -9.000000   
25%         19.000000       0.000000          1.000000          0.000000   
50%         36.000000       4.000000          5.000000          0.000000   
75%         58.000000      12.000000         16.000000          0.000000   
max       2542.000000     867.000000        841.000000        